In [1]:
from pylsl import StreamInlet, resolve_stream
import numpy as np
import joblib  # Used for loading sklearn models
import sys
import os
#import torch

sys.path.append('./src/processing')
from preprocessing import *

sys.path.append('./models')
from eegconformer import EEGConformer

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
local


In [2]:
models_dir = './models/trained/'
results_dir = './results/'

# Configuration
srate = 160  #Sampling rate of the EEG data
epoch_length_sec = 5  # Length of the desired sample in seconds
samples_needed = srate * epoch_length_sec  # Number of samples needed for ~5 seconds
# Manually define from eegbci dataset
channel_names = ['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FT8', 'T7', 'T8', 'T9', 'T10', 'TP7', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']


In [3]:

# Try resolving the EEG stream.
print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG')

# Assuming numpy is already imported.
# Assuming you have access to the inlet after resolving the stream.
inlet = StreamInlet(streams[0])

# Function to collect a single chunk.
def collect_one_chunk(inlet):
    while True:
        # Attempt to pull a chunk from the stream.
        chunk, timestamps = inlet.pull_chunk()

        if chunk:
            # If a chunk is received, save it and exit the function.
            print(f"Received chunk with timestamps: {timestamps[0]} to {timestamps[-1]}")
            np.save('chunk.npy', chunk)  # Ensure numpy is imported.
            print("Chunk saved.")
            break  # Exit the loop after saving the chunk.

# Call the function to start collecting.
collect_one_chunk(inlet)


Looking for an EEG stream...


2024-03-13 00:04:41.361 (   4.929s) [            9D67]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-13 00:04:41.361 (   4.929s) [            9D67]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-13 00:04:41.361 (   4.929s) [            9D67]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-03-13 00:04:41.361 (   4.929s) [            9D67]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-13 00:04:41.361 (   4.929s) [            9D67]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-03-13 00:04:41.361 (   4.929s) [            9D67]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-13 00:04:41.361 (   4.929s) [            9D67]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-03-13 00:04:41.361 (   4.929s) [            9D67]      netinterfaces.cpp:91    I

In [ ]:

# Configuration
srate = 160  # Sampling rate of the EEG data, adjust as needed
epoch_length_sec = 5  # Length of the desired sample in seconds
samples_needed = srate * epoch_length_sec  # Number of samples needed for ~5 seconds

print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG')
inlet = StreamInlet(streams[0])

def collect_and_save_single_sample(inlet, samples_needed):
    buffer = []  # Initialize the buffer to hold collected samples
    timestamps = []  # To store timestamps of each sample

    while len(buffer) < samples_needed:
        # Continuously pull samples
        sample, timestamp = inlet.pull_sample()
        if sample:
            buffer.append(sample)  # Add the sample to the buffer
            timestamps.append(timestamp)  # Add the timestamp

        if len(buffer) >= samples_needed:
            # Once we have enough samples, save and exit
            np.save('sample.npy', np.array(buffer))  # Save the buffer as a numpy file
            print(f"Saved ~{epoch_length_sec}-second sample with {len(buffer)} samples.")
            return  # Exit the function, effectively stopping data collection

# Call the function to collect, save, and then stop
collect_and_save_single_sample(inlet, samples_needed)


In [ ]:

print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG')
inlet = StreamInlet(streams[0])

def collect_and_save_single_sample(inlet, samples_needed):
    '''
    Test function to collect and save a single sample of EEG data

    :param inlet: The LSL StreamInlet object
    :param samples_needed: The number of samples to collect
    :return: None
    '''
    buffer = []  # Initialize the buffer to hold collected samples
    timestamps = []  # To store timestamps of each sample

    while len(buffer) < samples_needed:
        # Continuously pull samples
        sample, timestamp = inlet.pull_sample()
        if sample:
            buffer.append(sample)  # Add the sample to the buffer
            timestamps.append(timestamp)  # Add the timestamp

        if len(buffer) >= samples_needed:
            # Once we have enough samples, save and exit
            np.save('sample.npy', np.array(buffer))  # Save the buffer as a numpy file
            print(f"Saved ~{epoch_length_sec}-second sample with {len(buffer)} samples.")
            return  # Exit the function, effectively stopping data collection

# Call the function to collect, save, and then stop
collect_and_save_single_sample(inlet, samples_needed)
sys.exit()


In [ ]:
# # Test with one saved sample sent 
sample = np.load('sample.npy')
sample.shape

In [ ]:
preprocessed_data = preprocess_single_trial(sample, srate, channel_names)

In [ ]:
# csp + lda decode
model_path = os.path.join(models_dir, 'csp_lda.pkl')
loaded_model = joblib.load(model_path)
predicted_labels = loaded_model.predict(preprocessed_data)
predicted_labels

# Save result

In [ ]:
# csp + logistic regression decode
model_path = os.path.join(models_dir, 'csp_logistic.pkl')
loaded_model = joblib.load(model_path)
predicted_labels = loaded_model.predict(preprocessed_data)
predicted_labels

In [ ]:
# csp + svm decode
model_path = os.path.join(models_dir, 'csp_svm.pkl')
loaded_model = joblib.load(model_path)
predicted_labels = loaded_model.predict(preprocessed_data)
predicted_labels

In [ ]:
_, n_chans, n_times = preprocessed_data.shape

In [ ]:
# eeg_conformer decode
model = EEGConformer(
    n_outputs= 2,
    n_chans = n_chans,
    sfreq= srate,
    n_times = n_times,
    n_filters_time=40, 
    filter_time_length=25,
    pool_time_length=75,
    pool_time_stride=15,
    drop_prob=0.7,
    att_depth=3,
    att_heads=10,
    att_drop_prob=0.7,
    final_fc_length='auto', # could be 'auto' or int
    return_features=False, # returns the features before the last classification layer if True
    chs_info=None,
    input_window_seconds=None,
    add_log_softmax=True,
)

In [ ]:
loaded_model = os.path.join(models_dir, 'cross_subject_conformer.pth')
checkpoint = torch.load(loaded_model)

In [ ]:
model.load_state_dict(checkpoint)

In [ ]:
def online_decode(inlet, samples_per_epoch, loaded_model, srate, channel_names, results_dir):
    """
    Continuously pull samples from the LSL stream and decode them.
    """
    buffer = []  # Initialize buffer for accumulating samples
    pred_hist = []  # History of predictions

    while True:
        # Pull sample from LSL stream
        sample, timestamp = inlet.pull_sample()
        if sample:  # Ensure sample is not None
            buffer.append(sample)

        # Check if buffer has enough samples to form an epoch
        if len(buffer) >= samples_per_epoch:
            epoch = np.array(buffer[:samples_per_epoch])
            buffer = buffer[samples_per_epoch:]  # Remove the processed samples from the buffer

            # Decode the epoch
            prediction = decode_sample(epoch, loaded_model, srate, channel_names)
            pred_hist.append((timestamp, prediction))  # Append the prediction and its timestamp to the history
            print(f"Timestamp: {timestamp}, Prediction: {prediction}")

        # Condition to stop after decoding 3 epochs
        if len(pred_hist) >= 3:
            # Save the results and exit the loop
            np.save(os.path.join(results_dir, 'results.npy'), np.array(pred_hist))
            print("Saved 3 epochs and their predictions.")
            break  # Exit the while loop

def decode_sample(epoch, loaded_model, srate, channel_names):
    """
    Process and decode an epoch of EEG data.
    """
    preprocessed_epoch = preprocess_single_trial(epoch, srate, channel_names)  # Ensure this function exists and works as intended
    prediction = loaded_model.predict(preprocessed_epoch)  # Fixed to use the correct variable
    return prediction

# Example of calling online_decode (make sure all the required variables are defined)
# online_decode(inlet, samples_per_epoch, loaded_model, srate, channel_names, results_dir)


In [ ]:
model_path = os.path.join(models_dir, 'csp_logistic.pkl')
loaded_model = joblib.load(model_path)

print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG')
online_decode(inlet, samples_per_epoch, loaded_model, srate, channel_names, results_dir)

In [ ]:
samples_per_epoch = srate * epoch_length_sec

# Resolve the stream
print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG', 'name', 'BioSemi')
#inlet = StreamInlet(streams[0])

# Initialize a buffer for accumulating samples
buffer = []

model_path = os.path.join(models_dir, 'csp_logistic.pkl')
loaded_model = joblib.load(model_path)

def decode_sample(epoch, loaded_model, srate, channel_names):
    """
    Process and decode an epoch of EEG data.
    """
    preprocessed_epoch = preprocess_single_trial(epoch, srate, channel_names)
    predicted_labels = loaded_model.predict(preprocessed_data)
    return prediction

def online_decode(inlet):
    """
    Continuously pull samples from the LSL stream and decode them.
    """
    pred_hist = []
    while True:
        # Pull sample from LSL stream
        sample, timestamp = inlet.pull_sample()
        buffer.append(sample)
        
        # Check if buffer has enough samples to form an epoch
        if len(buffer) >= samples_per_epoch:
            epoch = np.array(buffer[:samples_per_epoch])  
            buffer = buffer[samples_per_epoch:]  
            
            # Decode the epoch
            prediction = decode_sample(epoch, loaded_model, srate, channel_names)
            pred_hist.append((timestamp, prediction))
            print(f"Timestamp: {timestamp}, Prediction: {prediction}")

        if len(pred_hist) >= 3:
            # Save the results
            np.save(os.path.join(results_dir, 'results.npy'), np.array(pred_hist))
            return

